In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=9855adbb0c7e2f525eed731564dfe923ff5e3cafab70beb541f8777292631e5c
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [ ]:
from google.colab import files
import zipfile
import os
import pandas as pd
import glob

uploaded = files.upload()
zip_path = next(iter(uploaded))
extract_folder = "/content/csvs"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

archivos = glob.glob(f"{extract_folder}/**/*.csv", recursive=True)

dataframes_validos = []
columnas_clave = ['Producto', 'Descripción de careacterística', 'Cant. movida', 'Hueco']

for archivo in archivos:
    try:
        df = pd.read_csv(archivo, sep=";", encoding="latin1")
        if all(col in df.columns for col in columnas_clave):
            dataframes_validos.append(df)
            print(f" {os.path.basename(archivo)} cargado")
        else:
            print(f" Columnas faltantes en {os.path.basename(archivo)}")
    except Exception as e:
        print(f" Error leyendo {os.path.basename(archivo)}: {e}")

if not dataframes_validos:
    raise ValueError("No se cargaron archivos válidos")

df_total = pd.concat(dataframes_validos, ignore_index=True)

df_total = df_total.rename(columns={
    'Producto': 'producto',
    'Descripción de careacterística': 'talla',
    'Cant. movida': 'cantidad',
    'Hueco': 'ubicacion'
})

for col in ['producto', 'talla', 'cantidad', 'ubicacion']:
    if col not in df_total.columns:
        raise ValueError(f" Falta la columna requerida: '{col}'")

df_total['producto'] = df_total['producto'].astype(str).str.upper().str.strip()
df_total['talla'] = df_total['talla'].astype(str).str.upper().str.strip()
df_total['ubicacion'] = df_total['ubicacion'].astype(str).str.upper().str.strip()
df_total['cantidad'] = pd.to_numeric(df_total['cantidad'], errors='coerce').fillna(0)

df_total['ubicacion'] = df_total['ubicacion'].replace({
    'LOGINSER CENTRAL': 'LOGINSER',
    'LOGINSER': 'LOGINSER',
    'TIENDA': 'TIENDA',
    'TRANSITO LOGINSER': 'LOGINSER',
    'CENTRO': 'CENTRO',
    'PEDIDOS CENTRO': 'CENTRO',
    'TRANSITO CENTRO': 'CENTRO'
})

df_total = df_total[df_total['ubicacion'].isin(['TIENDA', 'LOGINSER', 'CENTRO'])]

from IPython.display import display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

display(df_total)

df_agrupado = df_total.groupby(['producto', 'talla', 'ubicacion'])['cantidad'].sum().reset_index()
display(df_agrupado)

excel_output_path = "/content/df_agrupado.xlsx"
df_agrupado.to_excel(excel_output_path, index=False)
print(f"Excel file saved to: {excel_output_path}")

try:
    files.download(excel_output_path)
except FileNotFoundError:
    print(f"Error: The file {excel_output_path} was not found. Please ensure the Excel file was generated successfully.")

from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 11)
        self.cell(0, 10, 'Resumen agrupado de albaranes', ln=True, align='C')
        self.ln(4)

        self.set_font("Arial", 'B', 9)
        headers = ['producto', 'talla', 'ubicacion', 'cantidad']
        for i, header in enumerate(headers):
            self.cell(col_widths[i], 8, header.upper(), border=1, align='C')
        self.ln()

col_widths = [180, 40, 35, 22]
line_height = 8

pdf = PDF(orientation='L', unit='mm', format='A4')
pdf.add_page()
pdf.set_font("Arial", '', 8)

for _, row in df_agrupado.iterrows():
    producto_lines = len(pdf.multi_cell(col_widths[0], line_height, str(row['producto']), border=0, split_only=True))
    row_height = producto_lines * line_height

    if pdf.get_y() + row_height > 190:
        pdf.add_page()

    y_before = pdf.get_y()
    x = pdf.get_x()

    pdf.multi_cell(col_widths[0], line_height, str(row['producto']), border=1)
    y_after = pdf.get_y()
    actual_row_height = y_after - y_before

    pdf.set_xy(x + col_widths[0], y_before)
    pdf.multi_cell(col_widths[1], actual_row_height, str(row['talla']), border=1, align='C')

    pdf.set_xy(x + col_widths[0] + col_widths[1], y_before)
    pdf.multi_cell(col_widths[2], actual_row_height, str(row['ubicacion']), border=1, align='C')

    pdf.set_xy(x + col_widths[0] + col_widths[1] + col_widths[2], y_before)
    pdf.multi_cell(col_widths[3], actual_row_height, str(int(row['cantidad'])), border=1, align='C')

    pdf.set_y(y_before + actual_row_height)

pdf_output_path = "/content/df_agrupado.pdf"
pdf.output(pdf_output_path)

files.download(pdf_output_path)


Saving 19.zip to 19.zip
✅ ExportedData (15).csv cargado
✅ ExportedData (33).csv cargado
✅ ExportedData (17).csv cargado
✅ ExportedData (26).csv cargado
✅ ExportedData (27).csv cargado
✅ ExportedData (10).csv cargado
✅ ExportedData (20).csv cargado
✅ ExportedData (7).csv cargado
✅ ExportedData (25).csv cargado
✅ ExportedData (4).csv cargado
✅ ExportedData (24).csv cargado
✅ ExportedData (1).csv cargado
✅ ExportedData (23).csv cargado
✅ ExportedData (29).csv cargado
✅ ExportedData (9).csv cargado
✅ ExportedData (28).csv cargado
✅ ExportedData (18).csv cargado
✅ ExportedData (8).csv cargado
✅ ExportedData (13).csv cargado
✅ ExportedData (14).csv cargado
✅ ExportedData.csv cargado
✅ ExportedData (6).csv cargado
✅ ExportedData (5).csv cargado
✅ ExportedData (22).csv cargado
✅ ExportedData (30).csv cargado
✅ ExportedData (32).csv cargado
✅ ExportedData (21).csv cargado
✅ ExportedData (2).csv cargado
✅ ExportedData (31).csv cargado
✅ ExportedData (3).csv cargado
✅ ExportedData (11).csv cargad

,Línea,producto,talla,Referencia de almacén,Valor atributos,cantidad,Unidad,ubicacion,Centro de costos
0,20,19439-113803 - ZAPATILLAS ASICS NOVABLAST 5 RO...,"TALLA: 40,5 - EUR",1011B974-600-405,NaN,1,Unidad,LOGINSER,NaN
1,10,19439-107654 - ZAPATILLAS ASICS NOVABLAST 5 RO...,"TALLA: 46,5 - EUR",1011B974-600-465,NaN,1,Unidad,LOGINSER,NaN
2,20,19635-109231 - CALCETINES ASICS PERFORMANCE RU...,TALLA: II,3013B199-701-II,NaN,3,Unidad,LOGINSER,NaN
3,10,19635-109232 - CALCETINES ASICS PERFORMANCE RU...,TALLA: III,3013B199-701-III,NaN,6,Unidad,LOGINSER,NaN
4,80,19439-107646 - ZAPATILLAS ASICS NOVABLAST 5 RO...,"TALLA: 41,5 - EUR",1011B974-600-415,NaN,1,Unidad,LOGINSER,NaN
5,10,19439-107647 - ZAPATILLAS ASICS NOVABLAST 5 RO...,TALLA: 42 - EUR,1011B974-600-42,NaN,2,Unidad,LOGINSER,NaN
6,90,19439-107648 - ZAPATILLAS ASICS NOVABLAST 5 RO...,"TALLA: 42,5 - EUR",1011B974-600-425,NaN,2,Unidad,LOGINSER,NaN
7,100,19439-107649 - ZAPATILLAS ASICS NOVABLAST 5 RO...,"TALLA: 43,5 - EUR",1011B974-600-435,NaN,3,Unidad,LOGINSER,NaN
8,70,19439-107650 - ZAPATILLAS ASICS NOVABLAST 5 RO...,TALLA: 44 - EUR,1011B974-600-44,NaN,3,Unidad,LOGINSER,NaN
9,60,19439-107651 - ZAPATILLAS ASICS NOVABLAST 5 RO...,"TALLA: 44,5 - EUR",1011B974-600-445,NaN,2,Unidad,LOGINSER,NaN


,producto,talla,ubicacion,cantidad
0,10220-51564 - ZAPATILLAS BROOKS ADDICTION GTS ...,TALLA: 43 - EUR,TIENDA,2
1,10736-53555 - MANGUITOS CASTELLI PRO SEAMLESS ...,TALLA: S/M,TIENDA,1
2,11438 - CINTA COMPRESSPORT ON/OFF BLANCO,NAN,TIENDA,5
3,11445 - CINTA FINA COMPRESSPORT ON/OFF NEGRO,NAN,TIENDA,5
4,11618-56979 - CULOTE Q36.5 GREGARIUS ESSENTIAL...,TALLA: M,TIENDA,1
5,11618-58660 - CULOTE Q36.5 GREGARIUS ESSENTIAL...,TALLA: L,TIENDA,1
6,11834-58130 - GUANTES CASTELLI ARENBERG GEL 2 ...,TALLA: L,TIENDA,1
7,12623-63054 - CHAQUETA CASTELLI PERFETTO ROS 2...,TALLA: L,TIENDA,2
8,12628-63061 - GUANTES CASTELLI PERFETTO MAX NEGRO,TALLA: S,TIENDA,1
9,12628-63062 - GUANTES CASTELLI PERFETTO MAX NEGRO,TALLA: M,TIENDA,1


Excel file saved to: /content/df_agrupado.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>